In [1]:
import sys
sys.path.insert(0, "../..")

In [2]:
import torch
from torch import nn
from torchvision import models
from copy import deepcopy
import os

from nn_extrapolation import AcceleratedSGD
from nn_utils import *

In [3]:
trainer = Trainer(
    device="cuda:2",
    loss_fn=nn.NLLLoss(reduction="mean"),
    val_loss_fn=nn.NLLLoss(reduction="sum"),
)

In [4]:
dl = load_dataset(
    dataset="CIFAR10",
    root=os.path.join("/tmp", os.environ["USER"], "CIFAR"),
    augmentation=transforms.RandomAffine(10, scale=(0.9, 1.1), translate=(0.2, 0.2)),
    validation_split=0.2,
    batch_size=128,
    num_workers=10,
)

Files already downloaded and verified
Files already downloaded and verified


In [5]:
model = models.vgg16(pretrained=False)
model.classifier[6] = nn.Linear(4096, 10)
model.classifier.add_module("7", nn.LogSoftmax(-1))

model.to(trainer.device)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [6]:
sum(param.numel() for param in model.parameters())

134301514

In [6]:
trainer.validation(model, dl["valid"])

(0.1004, 2.3031887786865233)

## Momentum

### Lambda = 1e-16

In [24]:
optimizer = AcceleratedSGD(model.parameters(), 1e-2, k=10, momentum=0.9, weight_decay=1e-5, lambda_=1e-16)
logger = Logger("vgg_log_augmentation_lambda=1e-16.txt.no_resizing")

In [ ]:
epochs = 30

for epoch in range(epochs):
    train_loss = trainer.train_epoch(model, optimizer, dl["train"])
    optimizer.finish_epoch()
    val_acc, val_loss = trainer.validation(model, dl["valid"])
    logger.log("Epoch", epoch+1, "|", 
          f"Training loss: {train_loss:.4f}, validation accuracy: {val_acc:.4f}, validation loss: {val_loss:.4f}")

100%|██████████| 313/313 [00:41<00:00,  7.62it/s, loss=2.0714]
Epoch 1 | Training loss: 2.0714, validation accuracy: 0.3237, validation loss: 1.7468
100%|██████████| 313/313 [00:42<00:00,  7.39it/s, loss=1.7321]
Epoch 2 | Training loss: 1.7321, validation accuracy: 0.3998, validation loss: 1.5568
100%|██████████| 313/313 [00:42<00:00,  7.43it/s, loss=1.5215]
Epoch 3 | Training loss: 1.5215, validation accuracy: 0.4877, validation loss: 1.3630
100%|██████████| 313/313 [00:42<00:00,  7.45it/s, loss=1.3754]
Epoch 4 | Training loss: 1.3754, validation accuracy: 0.5446, validation loss: 1.2703
100%|██████████| 313/313 [00:42<00:00,  7.36it/s, loss=1.2239]
Epoch 5 | Training loss: 1.2239, validation accuracy: 0.6043, validation loss: 1.0995
 21%|██▏       | 67/313 [00:10<00:30,  8.08it/s, loss=1.1146]

In [ ]:
train_score = trainer.validation(model, dl["train"])
valid_score = trainer.validation(model, dl["valid"])
logger.log("Train:", train_score)
logger.log("Valid:", valid_score)

In [ ]:
optimizer.param_groups[0]["method"] = "RNA"
logger.log("RNA")
model_acc = deepcopy(model)
optimizer.accelerate()
optimizer.store_parameters([model_acc.parameters()])

In [ ]:
model_acc.to(trainer.device)
train_score = trainer.validation(model_acc, dl["train"])
valid_score = trainer.validation(model_acc, dl["valid"])
logger.log("Train:", train_score)
logger.log("Valid:", valid_score)

In [ ]:
optimizer.param_groups[0]["method"] = "RRE"
logger.log("RRE")
model_acc = deepcopy(model)
optimizer.accelerate()
optimizer.store_parameters([model_acc.parameters()])

In [ ]:
model_acc.to(trainer.device)
train_score = trainer.validation(model_acc, dl["train"])
valid_score = trainer.validation(model_acc, dl["valid"])
logger.log("Train:", train_score)
logger.log("Valid:", valid_score)

In [ ]:
optimizer.param_groups[0]["lr"] = 1e-3

In [ ]:
epochs = 30

for epoch in range(epochs):
    train_loss = trainer.train_epoch(model, optimizer, dl["train"])
    optimizer.finish_epoch()
    val_acc, val_loss = trainer.validation(model, dl["valid"])
    logger.log("Epoch", epoch+1, "|", 
          f"Training loss: {train_loss:.4f}, validation accuracy: {val_acc:.4f}, validation loss: {val_loss:.4f}")

In [ ]:
train_score = trainer.validation(model, dl["train"])
valid_score = trainer.validation(model, dl["valid"])
logger.log("Train:", train_score)
logger.log("Valid:", valid_score)

In [ ]:
optimizer.param_groups[0]["method"] = "RNA"
logger.log("RNA")
model_acc = deepcopy(model)
optimizer.accelerate()
optimizer.store_parameters([model_acc.parameters()])

In [ ]:
model_acc.to(trainer.device)
train_score = trainer.validation(model_acc, dl["train"])
valid_score = trainer.validation(model_acc, dl["valid"])
logger.log("Train:", train_score)
logger.log("Valid:", valid_score)

In [ ]:
optimizer.param_groups[0]["method"] = "RRE"
logger.log("RRE")
model_acc = deepcopy(model)
optimizer.accelerate()
optimizer.store_parameters([model_acc.parameters()])

In [ ]:
model_acc.to(trainer.device)
train_score = trainer.validation(model_acc, dl["train"])
valid_score = trainer.validation(model_acc, dl["valid"])
logger.log("Train:", train_score)
logger.log("Valid:", valid_score)

In [ ]:
exit

### Lambda = 1e-8

In [8]:
optimizer = AcceleratedSGD(model.parameters(), 1e-2, k=10, momentum=0.9, weight_decay=1e-5, lambda_=1e-8)
logger = Logger("vgg_log_augmentation.txt.no_resizing")

In [9]:
epochs = 30

for epoch in range(epochs):
    train_loss = trainer.train_epoch(model, optimizer, dl["train"])
    optimizer.finish_epoch()
    val_acc, val_loss = trainer.validation(model, dl["valid"])
    logger.log("Epoch", epoch+1, "|", 
          f"Training loss: {train_loss:.4f}, validation accuracy: {val_acc:.4f}, validation loss: {val_loss:.4f}")

100%|██████████| 313/313 [00:39<00:00,  7.88it/s, loss=2.1029]
Epoch 1 | Training loss: 2.1029, validation accuracy: 0.2809, validation loss: 1.8211
100%|██████████| 313/313 [00:41<00:00,  7.58it/s, loss=1.7491]
Epoch 2 | Training loss: 1.7491, validation accuracy: 0.4048, validation loss: 1.5428
100%|██████████| 313/313 [00:41<00:00,  7.51it/s, loss=1.5513]
Epoch 3 | Training loss: 1.5513, validation accuracy: 0.4785, validation loss: 1.4029
100%|██████████| 313/313 [00:41<00:00,  7.45it/s, loss=1.3674]
Epoch 4 | Training loss: 1.3674, validation accuracy: 0.5529, validation loss: 1.2307
100%|██████████| 313/313 [00:42<00:00,  7.44it/s, loss=1.2373]
Epoch 5 | Training loss: 1.2373, validation accuracy: 0.6244, validation loss: 1.0775
100%|██████████| 313/313 [00:42<00:00,  7.40it/s, loss=1.0975]
Epoch 6 | Training loss: 1.0975, validation accuracy: 0.6733, validation loss: 0.9169
100%|██████████| 313/313 [00:42<00:00,  7.36it/s, loss=1.0225]
Epoch 7 | Training loss: 1.0225, validation

In [10]:
train_score = trainer.validation(model, dl["train"])
valid_score = trainer.validation(model, dl["valid"])
logger.log("Train:", train_score)
logger.log("Valid:", valid_score)

Train: (0.89985, 0.298917316031456)
Valid: (0.8464, 0.48071938285827637)


In [11]:
optimizer.param_groups[0]["method"] = "RNA"
logger.log("RNA")
model_acc = deepcopy(model)
optimizer.accelerate()
optimizer.store_parameters([model_acc.parameters()])

RNA


In [12]:
model_acc.to(trainer.device)
train_score = trainer.validation(model_acc, dl["train"])
valid_score = trainer.validation(model_acc, dl["valid"])
logger.log("Train:", train_score)
logger.log("Valid:", valid_score)

Train: (0.9261, 0.21667036831378936)
Valid: (0.8685, 0.41548775596618653)


In [13]:
optimizer.param_groups[0]["method"] = "RRE"
logger.log("RRE")
model_acc = deepcopy(model)
optimizer.accelerate()
optimizer.store_parameters([model_acc.parameters()])

RRE


In [14]:
model_acc.to(trainer.device)
train_score = trainer.validation(model_acc, dl["train"])
valid_score = trainer.validation(model_acc, dl["valid"])
logger.log("Train:", train_score)
logger.log("Valid:", valid_score)

Train: (0.927375, 0.21593683211803436)
Valid: (0.8685, 0.415491977596283)


In [15]:
optimizer.param_groups[0]["lr"] = 1e-3

In [ ]:
epochs = 30

for epoch in range(epochs):
    train_loss = trainer.train_epoch(model, optimizer, dl["train"])
    optimizer.finish_epoch()
    val_acc, val_loss = trainer.validation(model, dl["valid"])
    logger.log("Epoch", epoch+1, "|", 
          f"Training loss: {train_loss:.4f}, validation accuracy: {val_acc:.4f}, validation loss: {val_loss:.4f}")

100%|██████████| 313/313 [00:42<00:00,  7.43it/s, loss=0.2174]
Epoch 1 | Training loss: 0.2174, validation accuracy: 0.8648, validation loss: 0.4610
100%|██████████| 313/313 [00:43<00:00,  7.25it/s, loss=0.1859]
Epoch 2 | Training loss: 0.1859, validation accuracy: 0.8659, validation loss: 0.4624
100%|██████████| 313/313 [00:42<00:00,  7.33it/s, loss=0.1738]
Epoch 3 | Training loss: 0.1738, validation accuracy: 0.8666, validation loss: 0.4674
100%|██████████| 313/313 [00:42<00:00,  7.35it/s, loss=0.1672]
Epoch 4 | Training loss: 0.1672, validation accuracy: 0.8670, validation loss: 0.4695
100%|██████████| 313/313 [00:43<00:00,  7.28it/s, loss=0.1630]
Epoch 5 | Training loss: 0.1630, validation accuracy: 0.8644, validation loss: 0.4796
100%|██████████| 313/313 [00:42<00:00,  7.30it/s, loss=0.1535]
Epoch 6 | Training loss: 0.1535, validation accuracy: 0.8692, validation loss: 0.4818
100%|██████████| 313/313 [00:43<00:00,  7.27it/s, loss=0.1508]
Epoch 7 | Training loss: 0.1508, validation

In [ ]:
train_score = trainer.validation(model, dl["train"])
valid_score = trainer.validation(model, dl["valid"])
logger.log("Train:", train_score)
logger.log("Valid:", valid_score)

In [ ]:
optimizer.param_groups[0]["method"] = "RNA"
logger.log("RNA")
model_acc = deepcopy(model)
optimizer.accelerate()
optimizer.store_parameters([model_acc.parameters()])

In [ ]:
model_acc.to(trainer.device)
train_score = trainer.validation(model_acc, dl["train"])
valid_score = trainer.validation(model_acc, dl["valid"])
logger.log("Train:", train_score)
logger.log("Valid:", valid_score)

In [ ]:
optimizer.param_groups[0]["method"] = "RRE"
logger.log("RRE")
model_acc = deepcopy(model)
optimizer.accelerate()
optimizer.store_parameters([model_acc.parameters()])

In [ ]:
model_acc.to(trainer.device)
train_score = trainer.validation(model_acc, dl["train"])
valid_score = trainer.validation(model_acc, dl["valid"])
logger.log("Train:", train_score)
logger.log("Valid:", valid_score)

In [ ]:
exit

### Lambda = 1e-4

In [6]:
optimizer = AcceleratedSGD(model.parameters(), 1e-2, k=10, momentum=0.9, weight_decay=1e-5, lambda_=1e-4)
logger = Logger("vgg_log_augmentation_lambda=1e-4.txt.no_resizing")

In [7]:
epochs = 30

for epoch in range(epochs):
    train_loss = trainer.train_epoch(model, optimizer, dl["train"])
    optimizer.finish_epoch()
    val_acc, val_loss = trainer.validation(model, dl["valid"])
    logger.log("Epoch", epoch+1, "|", 
          f"Training loss: {train_loss:.4f}, validation accuracy: {val_acc:.4f}, validation loss: {val_loss:.4f}")

100%|██████████| 313/313 [00:38<00:00,  8.06it/s, loss=2.0846]
Epoch 1 | Training loss: 2.0846, validation accuracy: 0.3103, validation loss: 1.7833
100%|██████████| 313/313 [00:40<00:00,  7.81it/s, loss=1.7563]
Epoch 2 | Training loss: 1.7563, validation accuracy: 0.3637, validation loss: 1.5992
100%|██████████| 313/313 [00:40<00:00,  7.68it/s, loss=1.5475]
Epoch 3 | Training loss: 1.5475, validation accuracy: 0.4515, validation loss: 1.5243
100%|██████████| 313/313 [00:40<00:00,  7.67it/s, loss=1.3973]
Epoch 4 | Training loss: 1.3973, validation accuracy: 0.5514, validation loss: 1.2665
100%|██████████| 313/313 [00:40<00:00,  7.65it/s, loss=1.2526]
Epoch 5 | Training loss: 1.2526, validation accuracy: 0.6072, validation loss: 1.0765
100%|██████████| 313/313 [00:40<00:00,  7.71it/s, loss=1.1292]
Epoch 6 | Training loss: 1.1292, validation accuracy: 0.6489, validation loss: 0.9864
100%|██████████| 313/313 [00:40<00:00,  7.74it/s, loss=1.0385]
Epoch 7 | Training loss: 1.0385, validation

In [8]:
train_score = trainer.validation(model, dl["train"])
valid_score = trainer.validation(model, dl["valid"])
logger.log("Train:", train_score)
logger.log("Valid:", valid_score)

Train: (0.9026, 0.2861096531867981)
Valid: (0.8406, 0.5020481191635132)


In [9]:
optimizer.param_groups[0]["method"] = "RNA"
logger.log("RNA")
model_acc = deepcopy(model)
optimizer.accelerate()
optimizer.store_parameters([model_acc.parameters()])

RNA


In [10]:
model_acc.to(trainer.device)
train_score = trainer.validation(model_acc, dl["train"])
valid_score = trainer.validation(model_acc, dl["valid"])
logger.log("Train:", train_score)
logger.log("Valid:", valid_score)

Train: (0.924275, 0.222084708404541)
Valid: (0.8641, 0.41911623668670656)


In [11]:
optimizer.param_groups[0]["method"] = "RRE"
logger.log("RRE")
model_acc = deepcopy(model)
optimizer.accelerate()
optimizer.store_parameters([model_acc.parameters()])

RRE


In [12]:
model_acc.to(trainer.device)
train_score = trainer.validation(model_acc, dl["train"])
valid_score = trainer.validation(model_acc, dl["valid"])
logger.log("Train:", train_score)
logger.log("Valid:", valid_score)

Train: (0.9265, 0.2205881089925766)
Valid: (0.864, 0.41911824893951416)


In [13]:
optimizer.param_groups[0]["lr"] = 1e-3

In [14]:
epochs = 30

for epoch in range(epochs):
    train_loss = trainer.train_epoch(model, optimizer, dl["train"])
    optimizer.finish_epoch()
    val_acc, val_loss = trainer.validation(model, dl["valid"])
    logger.log("Epoch", epoch+1, "|", 
          f"Training loss: {train_loss:.4f}, validation accuracy: {val_acc:.4f}, validation loss: {val_loss:.4f}")

100%|██████████| 313/313 [00:41<00:00,  7.63it/s, loss=0.2165]
Epoch 1 | Training loss: 0.2165, validation accuracy: 0.8637, validation loss: 0.4547
100%|██████████| 313/313 [00:41<00:00,  7.51it/s, loss=0.1935]
Epoch 2 | Training loss: 0.1935, validation accuracy: 0.8624, validation loss: 0.4563
100%|██████████| 313/313 [00:41<00:00,  7.58it/s, loss=0.1816]
Epoch 3 | Training loss: 0.1816, validation accuracy: 0.8656, validation loss: 0.4630
100%|██████████| 313/313 [00:41<00:00,  7.62it/s, loss=0.1720]
Epoch 4 | Training loss: 0.1720, validation accuracy: 0.8674, validation loss: 0.4621
100%|██████████| 313/313 [00:40<00:00,  7.64it/s, loss=0.1658]
Epoch 5 | Training loss: 0.1658, validation accuracy: 0.8633, validation loss: 0.4824
100%|██████████| 313/313 [00:40<00:00,  7.65it/s, loss=0.1600]
Epoch 6 | Training loss: 0.1600, validation accuracy: 0.8662, validation loss: 0.4811
100%|██████████| 313/313 [00:40<00:00,  7.67it/s, loss=0.1567]
Epoch 7 | Training loss: 0.1567, validation

In [15]:
train_score = trainer.validation(model, dl["train"])
valid_score = trainer.validation(model, dl["valid"])
logger.log("Train:", train_score)
logger.log("Valid:", valid_score)

Train: (0.97035, 0.08675295971035957)
Valid: (0.8669, 0.559516147327423)


In [16]:
optimizer.param_groups[0]["method"] = "RNA"
logger.log("RNA")
model_acc = deepcopy(model)
optimizer.accelerate()
optimizer.store_parameters([model_acc.parameters()])

RNA


In [17]:
model_acc.to(trainer.device)
train_score = trainer.validation(model_acc, dl["train"])
valid_score = trainer.validation(model_acc, dl["valid"])
logger.log("Train:", train_score)
logger.log("Valid:", valid_score)

Train: (0.969525, 0.08780566491484643)
Valid: (0.8729, 0.5333709650039673)


In [18]:
optimizer.param_groups[0]["method"] = "RRE"
logger.log("RRE")
model_acc = deepcopy(model)
optimizer.accelerate()
optimizer.store_parameters([model_acc.parameters()])

RRE


In [19]:
model_acc.to(trainer.device)
train_score = trainer.validation(model_acc, dl["train"])
valid_score = trainer.validation(model_acc, dl["valid"])
logger.log("Train:", train_score)
logger.log("Valid:", valid_score)

Train: (0.96925, 0.08957459921240807)
Valid: (0.8729, 0.5333706748962402)


## Epoch average

In [5]:
model = models.vgg16(pretrained=False)
model.classifier[6] = nn.Linear(4096, 10)
model.classifier.add_module("7", nn.LogSoftmax(-1))

model.to(device)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [6]:
validation(model, valid_loader)

(0.1077, 2.3034106719970704)

In [7]:
log_file = open("vgg_log_augmentation_averaging.txt", "w")

In [8]:
optimizer = AcceleratedSGD(model.parameters(), 1e-2, k=10, momentum=0.9, weight_decay=1e-5, lambda_=1e-8, mode="epoch_avg")
loss_fn = nn.NLLLoss()

In [9]:
epochs = 30

for epoch in range(epochs):
    print("Epoch", epoch+1)
    loss_log = []
    train_epoch(loss_log)
    print(f"Training loss: {np.mean(loss_log):.4f}")
    optimizer.finish_epoch()
    val_acc, val_loss = validation(model, valid_loader)
    print(f"Validation accuracy: {val_acc:.4f}, validation loss: {val_loss:.4f}")
    print("Epoch", epoch+1, 
          f"Training loss: {np.mean(loss_log):.4f}, validation accuracy: {val_acc:.4f}, validation loss: {val_loss:.4f}",
          file=log_file, flush=True
         )

Epoch 1



Training loss: 2.1002



Validation accuracy: 0.2559, validation loss: 1.9581
Epoch 2



Training loss: 1.7542



Validation accuracy: 0.3842, validation loss: 1.5413
Epoch 3


...


Validation accuracy: 0.8404, validation loss: 0.4998
Epoch 29



Training loss: 0.3380



Validation accuracy: 0.8417, validation loss: 0.5059
Epoch 30



Training loss: 0.3273



Validation accuracy: 0.8469, validation loss: 0.4882


In [10]:
train_score = validation(model, train_loader)
valid_score = validation(model, valid_loader)
print("Train:", train_score)
print("Valid:", valid_score)
print("Train:", train_score, flush=True, file=log_file)
print("Valid:", valid_score, flush=True, file=log_file)


Train: (0.903475, 0.27089233884811403)
Valid: (0.8469, 0.4882191445350647)


In [11]:
model_acc = deepcopy(model)
optimizer.accelerate()
optimizer.store_parameters([model_acc.parameters()])

In [12]:
model_acc.to(device)
train_score = validation(model_acc, train_loader)
valid_score = validation(model_acc, valid_loader)

print("Train:", train_score)
print("Valid:", valid_score)
print("Train:", train_score, flush=True, file=log_file)
print("Valid:", valid_score, flush=True, file=log_file)


Train: (0.794, 0.6776266638755798)
Valid: (0.7897, 0.6686483863830567)


In [13]:
optimizer.param_groups[0]["lr"] = 1e-3

In [14]:
epochs = 30

for epoch in range(epochs):
    print("Epoch", epoch+1)
    loss_log = []
    train_epoch(loss_log)
    print(f"Training loss: {np.mean(loss_log):.4f}")
    optimizer.finish_epoch()
    val_acc, val_loss = validation(model, valid_loader)
    print(f"Validation accuracy: {val_acc:.4f}, validation loss: {val_loss:.4f}")
    print("Epoch", epoch+1, 
          f"Training loss: {np.mean(loss_log):.4f}, validation accuracy: {val_acc:.4f}, validation loss: {val_loss:.4f}",
          file=log_file, flush=True
         )

Epoch 1



Training loss: 0.2150



Validation accuracy: 0.8661, validation loss: 0.4644
Epoch 2



Training loss: 0.1947



Validation accuracy: 0.8668, validation loss: 0.4567
Epoch 3


...


Validation accuracy: 0.8740, validation loss: 0.5231
Epoch 29



Training loss: 0.1000



Validation accuracy: 0.8725, validation loss: 0.5357
Epoch 30



Training loss: 0.0956



Validation accuracy: 0.8725, validation loss: 0.5321


In [15]:
train_score = validation(model, train_loader)
valid_score = validation(model, valid_loader)
print("Train:", train_score)
print("Valid:", valid_score)
print("Train:", train_score, flush=True, file=log_file)
print("Valid:", valid_score, flush=True, file=log_file)


Train: (0.9695, 0.09016556673645973)
Valid: (0.8725, 0.5321015692710876)


In [16]:
model_acc = deepcopy(model)
optimizer.accelerate()
optimizer.store_parameters([model_acc.parameters()])

In [17]:
model_acc.to(device)
train_score = validation(model_acc, train_loader)
valid_score = validation(model_acc, valid_loader)

print("Train:", train_score)
print("Valid:", valid_score)
print("Train:", train_score, flush=True, file=log_file)
print("Valid:", valid_score, flush=True, file=log_file)


Train: (0.94195, 0.1775376051902771)
Valid: (0.8687, 0.41627861557006834)


## Epoch average, span = 100

In [7]:
model = models.vgg16(pretrained=False)
model.classifier[6] = nn.Linear(4096, 10)
model.classifier.add_module("7", nn.LogSoftmax(-1))

model.to(trainer.device)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [8]:
optimizer = AcceleratedSGD(model.parameters(), 1e-2, k=10, momentum=0.9, weight_decay=1e-5, lambda_=1e-8, 
                           mode="epoch_avg", avg_alpha=2 / (100 + 1))
logger = Logger("vgg_log_augmentation_span100.txt.no_resizing")

In [9]:
epochs = 30

for epoch in range(epochs):
    train_loss = trainer.train_epoch(model, optimizer, dl["train"])
    optimizer.finish_epoch()
    val_acc, val_loss = trainer.validation(model, dl["valid"])
    logger.log("Epoch", epoch+1, "|", 
          f"Training loss: {train_loss:.4f}, validation accuracy: {val_acc:.4f}, validation loss: {val_loss:.4f}")

100%|██████████| 313/313 [00:44<00:00,  7.00it/s, loss=2.0784]
Epoch 1 | Training loss: 2.0784, validation accuracy: 0.2824, validation loss: 1.8031
100%|██████████| 313/313 [00:46<00:00,  6.77it/s, loss=1.7458]
Epoch 2 | Training loss: 1.7458, validation accuracy: 0.4094, validation loss: 1.5733
100%|██████████| 313/313 [00:46<00:00,  6.70it/s, loss=1.5476]
Epoch 3 | Training loss: 1.5476, validation accuracy: 0.4922, validation loss: 1.3895
100%|██████████| 313/313 [00:46<00:00,  6.68it/s, loss=1.3608]
Epoch 4 | Training loss: 1.3608, validation accuracy: 0.5612, validation loss: 1.1960
100%|██████████| 313/313 [00:46<00:00,  6.68it/s, loss=1.2234]
Epoch 5 | Training loss: 1.2234, validation accuracy: 0.6230, validation loss: 1.0363
100%|██████████| 313/313 [00:46<00:00,  6.69it/s, loss=1.1015]
Epoch 6 | Training loss: 1.1015, validation accuracy: 0.6566, validation loss: 0.9850
100%|██████████| 313/313 [00:46<00:00,  6.67it/s, loss=1.0013]
Epoch 7 | Training loss: 1.0013, validation

In [10]:
train_score = trainer.validation(model, dl["train"])
valid_score = trainer.validation(model, dl["valid"])
logger.log("Train:", train_score)
logger.log("Valid:", valid_score)

Train: (0.902875, 0.28111403212547303)
Valid: (0.8398, 0.5249147163391114)


In [11]:
optimizer.param_groups[0]["method"] = "RNA"
logger.log("RNA")
model_acc = deepcopy(model)
optimizer.accelerate()
optimizer.store_parameters([model_acc.parameters()])

RNA


In [12]:
model_acc.to(trainer.device)
train_score = trainer.validation(model_acc, dl["train"])
valid_score = trainer.validation(model_acc, dl["valid"])
logger.log("Train:", train_score)
logger.log("Valid:", valid_score)

Train: (0.9261, 0.22179136071205138)
Valid: (0.8703, 0.3967107766151428)


In [13]:
optimizer.param_groups[0]["method"] = "RRE"
logger.log("RRE")
model_acc = deepcopy(model)
optimizer.accelerate()
optimizer.store_parameters([model_acc.parameters()])

RRE


In [14]:
model_acc.to(trainer.device)
train_score = trainer.validation(model_acc, dl["train"])
valid_score = trainer.validation(model_acc, dl["valid"])
logger.log("Train:", train_score)
logger.log("Valid:", valid_score)

Train: (0.92725, 0.2170490710258484)
Valid: (0.8703, 0.3967120982170105)


In [15]:
optimizer.param_groups[0]["lr"] = 1e-3

In [16]:
epochs = 30

for epoch in range(epochs):
    train_loss = trainer.train_epoch(model, optimizer, dl["train"])
    optimizer.finish_epoch()
    val_acc, val_loss = trainer.validation(model, dl["valid"])
    logger.log("Epoch", epoch+1, "|", 
          f"Training loss: {train_loss:.4f}, validation accuracy: {val_acc:.4f}, validation loss: {val_loss:.4f}")

100%|██████████| 313/313 [00:47<00:00,  6.55it/s, loss=0.2220]
Epoch 1 | Training loss: 0.2220, validation accuracy: 0.8661, validation loss: 0.4387
100%|██████████| 313/313 [00:48<00:00,  6.46it/s, loss=0.1906]
Epoch 2 | Training loss: 0.1906, validation accuracy: 0.8682, validation loss: 0.4503
100%|██████████| 313/313 [00:48<00:00,  6.50it/s, loss=0.1809]
Epoch 3 | Training loss: 0.1809, validation accuracy: 0.8688, validation loss: 0.4485
100%|██████████| 313/313 [00:47<00:00,  6.54it/s, loss=0.1734]
Epoch 4 | Training loss: 0.1734, validation accuracy: 0.8710, validation loss: 0.4519
100%|██████████| 313/313 [00:47<00:00,  6.58it/s, loss=0.1620]
Epoch 5 | Training loss: 0.1620, validation accuracy: 0.8685, validation loss: 0.4621
100%|██████████| 313/313 [00:47<00:00,  6.53it/s, loss=0.1613]
Epoch 6 | Training loss: 0.1613, validation accuracy: 0.8709, validation loss: 0.4672
100%|██████████| 313/313 [00:47<00:00,  6.55it/s, loss=0.1567]
Epoch 7 | Training loss: 0.1567, validation

In [17]:
train_score = trainer.validation(model, dl["train"])
valid_score = trainer.validation(model, dl["valid"])
logger.log("Train:", train_score)
logger.log("Valid:", valid_score)

Train: (0.96685, 0.09413220975399017)
Valid: (0.8725, 0.521118231010437)


In [18]:
optimizer.param_groups[0]["method"] = "RNA"
logger.log("RNA")
model_acc = deepcopy(model)
optimizer.accelerate()
optimizer.store_parameters([model_acc.parameters()])

RNA


In [19]:
model_acc.to(trainer.device)
train_score = trainer.validation(model_acc, dl["train"])
valid_score = trainer.validation(model_acc, dl["valid"])
logger.log("Train:", train_score)
logger.log("Valid:", valid_score)

Train: (0.96985, 0.08951034039855003)
Valid: (0.8757, 0.4972747452735901)


In [20]:
optimizer.param_groups[0]["method"] = "RRE"
logger.log("RRE")
model_acc = deepcopy(model)
optimizer.accelerate()
optimizer.store_parameters([model_acc.parameters()])

RRE


In [21]:
model_acc.to(trainer.device)
train_score = trainer.validation(model_acc, dl["train"])
valid_score = trainer.validation(model_acc, dl["valid"])
logger.log("Train:", train_score)
logger.log("Valid:", valid_score)

Train: (0.969675, 0.08840382719635964)
Valid: (0.8757, 0.49727380599975585)


## Epoch average, span = 50

In [22]:
model = models.vgg16(pretrained=False)
model.classifier[6] = nn.Linear(4096, 10)
model.classifier.add_module("7", nn.LogSoftmax(-1))

model.to(trainer.device)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [23]:
optimizer = AcceleratedSGD(model.parameters(), 1e-2, k=10, momentum=0.9, weight_decay=1e-5, lambda_=1e-8, 
                           mode="epoch_avg", avg_alpha=2 / (50 + 1))
logger = Logger("vgg_log_augmentation_span50.txt.no_resizing")

In [24]:
epochs = 30

for epoch in range(epochs):
    train_loss = trainer.train_epoch(model, optimizer, dl["train"])
    optimizer.finish_epoch()
    val_acc, val_loss = trainer.validation(model, dl["valid"])
    logger.log("Epoch", epoch+1, "|", 
          f"Training loss: {train_loss:.4f}, validation accuracy: {val_acc:.4f}, validation loss: {val_loss:.4f}")

100%|██████████| 313/313 [00:46<00:00,  6.78it/s, loss=2.0967]
Epoch 1 | Training loss: 2.0967, validation accuracy: 0.3072, validation loss: 1.8383
100%|██████████| 313/313 [00:47<00:00,  6.62it/s, loss=1.7519]
Epoch 2 | Training loss: 1.7519, validation accuracy: 0.4074, validation loss: 1.6071
100%|██████████| 313/313 [00:47<00:00,  6.66it/s, loss=1.5372]
Epoch 3 | Training loss: 1.5372, validation accuracy: 0.5034, validation loss: 1.3603
100%|██████████| 313/313 [00:46<00:00,  6.67it/s, loss=1.3780]
Epoch 4 | Training loss: 1.3780, validation accuracy: 0.5625, validation loss: 1.2158
100%|██████████| 313/313 [00:46<00:00,  6.67it/s, loss=1.2426]
Epoch 5 | Training loss: 1.2426, validation accuracy: 0.6115, validation loss: 1.0764
100%|██████████| 313/313 [00:46<00:00,  6.71it/s, loss=1.1235]
Epoch 6 | Training loss: 1.1235, validation accuracy: 0.6401, validation loss: 1.0222
100%|██████████| 313/313 [00:46<00:00,  6.68it/s, loss=1.0214]
Epoch 7 | Training loss: 1.0214, validation

In [25]:
train_score = trainer.validation(model, dl["train"])
valid_score = trainer.validation(model, dl["valid"])
logger.log("Train:", train_score)
logger.log("Valid:", valid_score)

Train: (0.886975, 0.32606319055557254)
Valid: (0.8393, 0.5027050075531005)


In [26]:
optimizer.param_groups[0]["method"] = "RNA"
logger.log("RNA")
model_acc = deepcopy(model)
optimizer.accelerate()
optimizer.store_parameters([model_acc.parameters()])

RNA


In [27]:
model_acc.to(trainer.device)
train_score = trainer.validation(model_acc, dl["train"])
valid_score = trainer.validation(model_acc, dl["valid"])
logger.log("Train:", train_score)
logger.log("Valid:", valid_score)

Train: (0.9272, 0.21609477653503417)
Valid: (0.8654, 0.40917909984588624)


In [28]:
optimizer.param_groups[0]["method"] = "RRE"
logger.log("RRE")
model_acc = deepcopy(model)
optimizer.accelerate()
optimizer.store_parameters([model_acc.parameters()])

RRE


In [29]:
model_acc.to(trainer.device)
train_score = trainer.validation(model_acc, dl["train"])
valid_score = trainer.validation(model_acc, dl["valid"])
logger.log("Train:", train_score)
logger.log("Valid:", valid_score)

Train: (0.92885, 0.2151413575410843)
Valid: (0.8654, 0.4091800615310669)


In [30]:
optimizer.param_groups[0]["lr"] = 1e-3

In [31]:
epochs = 30

for epoch in range(epochs):
    train_loss = trainer.train_epoch(model, optimizer, dl["train"])
    optimizer.finish_epoch()
    val_acc, val_loss = trainer.validation(model, dl["valid"])
    logger.log("Epoch", epoch+1, "|", 
          f"Training loss: {train_loss:.4f}, validation accuracy: {val_acc:.4f}, validation loss: {val_loss:.4f}")

100%|██████████| 313/313 [00:47<00:00,  6.55it/s, loss=0.2194]
Epoch 1 | Training loss: 0.2194, validation accuracy: 0.8632, validation loss: 0.4608
100%|██████████| 313/313 [00:48<00:00,  6.41it/s, loss=0.1882]
Epoch 2 | Training loss: 0.1882, validation accuracy: 0.8610, validation loss: 0.4759
100%|██████████| 313/313 [00:47<00:00,  6.52it/s, loss=0.1763]
Epoch 3 | Training loss: 0.1763, validation accuracy: 0.8660, validation loss: 0.4762
100%|██████████| 313/313 [00:48<00:00,  6.52it/s, loss=0.1667]
Epoch 4 | Training loss: 0.1667, validation accuracy: 0.8632, validation loss: 0.4853
100%|██████████| 313/313 [00:47<00:00,  6.54it/s, loss=0.1626]
Epoch 5 | Training loss: 0.1626, validation accuracy: 0.8678, validation loss: 0.4789
100%|██████████| 313/313 [00:47<00:00,  6.56it/s, loss=0.1570]
Epoch 6 | Training loss: 0.1570, validation accuracy: 0.8664, validation loss: 0.4762
100%|██████████| 313/313 [00:47<00:00,  6.54it/s, loss=0.1527]
Epoch 7 | Training loss: 0.1527, validation

In [32]:
train_score = trainer.validation(model, dl["train"])
valid_score = trainer.validation(model, dl["valid"])
logger.log("Train:", train_score)
logger.log("Valid:", valid_score)

Train: (0.9691, 0.09299993057549)
Valid: (0.8652, 0.5673083391189575)


In [33]:
optimizer.param_groups[0]["method"] = "RNA"
logger.log("RNA")
model_acc = deepcopy(model)
optimizer.accelerate()
optimizer.store_parameters([model_acc.parameters()])

RNA


In [34]:
model_acc.to(trainer.device)
train_score = trainer.validation(model_acc, dl["train"])
valid_score = trainer.validation(model_acc, dl["valid"])
logger.log("Train:", train_score)
logger.log("Valid:", valid_score)

Train: (0.97025, 0.08668203239440918)
Valid: (0.8698, 0.5264590911865235)


In [35]:
optimizer.param_groups[0]["method"] = "RRE"
logger.log("RRE")
model_acc = deepcopy(model)
optimizer.accelerate()
optimizer.store_parameters([model_acc.parameters()])

RRE


In [36]:
model_acc.to(trainer.device)
train_score = trainer.validation(model_acc, dl["train"])
valid_score = trainer.validation(model_acc, dl["valid"])
logger.log("Train:", train_score)
logger.log("Valid:", valid_score)

Train: (0.969775, 0.0866311824142933)
Valid: (0.8698, 0.5264550930023193)


## Epoch average, span = 10

In [6]:
model = models.vgg16(pretrained=False)
model.classifier[6] = nn.Linear(4096, 10)
model.classifier.add_module("7", nn.LogSoftmax(-1))

model.to(device)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [19]:
validation(model, valid_loader)

(0.1064, 2.302417967605591)

In [20]:
log_file = open("vgg_log_augmentation_averaging_span10.txt", "w")

In [21]:
optimizer = AcceleratedSGD(model.parameters(), 1e-2, k=10, momentum=0.9, weight_decay=1e-5, lambda_=1e-8, mode="epoch_avg", avg_alpha=2 / (10 + 1))
loss_fn = nn.NLLLoss()

In [22]:
epochs = 30

for epoch in range(epochs):
    print("Epoch", epoch+1)
    loss_log = []
    train_epoch(loss_log)
    print(f"Training loss: {np.mean(loss_log):.4f}")
    optimizer.finish_epoch()
    val_acc, val_loss = validation(model, valid_loader)
    print(f"Validation accuracy: {val_acc:.4f}, validation loss: {val_loss:.4f}")
    print("Epoch", epoch+1, 
          f"Training loss: {np.mean(loss_log):.4f}, validation accuracy: {val_acc:.4f}, validation loss: {val_loss:.4f}",
          file=log_file, flush=True
         )

Epoch 1



Training loss: 2.0916



Validation accuracy: 0.3147, validation loss: 1.7688
Epoch 2



Training loss: 1.7325



Validation accuracy: 0.4170, validation loss: 1.5160
Epoch 3


...


Validation accuracy: 0.8487, validation loss: 0.4893
Epoch 29



Training loss: 0.3300



Validation accuracy: 0.8319, validation loss: 0.5443
Epoch 30



Training loss: 0.3225



Validation accuracy: 0.8526, validation loss: 0.4737


In [23]:
train_score = validation(model, train_loader)
valid_score = validation(model, valid_loader)
print("Train:", train_score)
print("Valid:", valid_score)
print("Train:", train_score, flush=True, file=log_file)
print("Valid:", valid_score, flush=True, file=log_file)


Train: (0.9039, 0.2783135716676712)
Valid: (0.8526, 0.4736805819511414)


In [24]:
model_acc = deepcopy(model)
optimizer.accelerate()
optimizer.store_parameters([model_acc.parameters()])

In [25]:
model_acc.to(device)
train_score = validation(model_acc, train_loader)
valid_score = validation(model_acc, valid_loader)

print("Train:", train_score)
print("Valid:", valid_score)
print("Train:", train_score, flush=True, file=log_file)
print("Valid:", valid_score, flush=True, file=log_file)


Train: (0.92435, 0.22567135038375855)
Valid: (0.8707, 0.40380759258270266)


In [26]:
optimizer.param_groups[0]["lr"] = 1e-3

In [27]:
epochs = 30

for epoch in range(epochs):
    print("Epoch", epoch+1)
    loss_log = []
    train_epoch(loss_log)
    print(f"Training loss: {np.mean(loss_log):.4f}")
    optimizer.finish_epoch()
    val_acc, val_loss = validation(model, valid_loader)
    print(f"Validation accuracy: {val_acc:.4f}, validation loss: {val_loss:.4f}")
    print("Epoch", epoch+1, 
          f"Training loss: {np.mean(loss_log):.4f}, validation accuracy: {val_acc:.4f}, validation loss: {val_loss:.4f}",
          file=log_file, flush=True
         )

Epoch 1



Training loss: 0.2114



Validation accuracy: 0.8642, validation loss: 0.4549
Epoch 2



Training loss: 0.1894



Validation accuracy: 0.8674, validation loss: 0.4486
Epoch 3


...


Validation accuracy: 0.8718, validation loss: 0.5108
Epoch 29



Training loss: 0.1009



Validation accuracy: 0.8715, validation loss: 0.5146
Epoch 30



Training loss: 0.0980



Validation accuracy: 0.8697, validation loss: 0.5237


In [28]:
train_score = validation(model, train_loader)
valid_score = validation(model, valid_loader)
print("Train:", train_score)
print("Valid:", valid_score)
print("Train:", train_score, flush=True, file=log_file)
print("Valid:", valid_score, flush=True, file=log_file)


Train: (0.970025, 0.08667169282436371)
Valid: (0.8697, 0.5237287378311157)


In [29]:
model_acc = deepcopy(model)
optimizer.accelerate()
optimizer.store_parameters([model_acc.parameters()])

In [30]:
model_acc.to(device)
train_score = validation(model_acc, train_loader)
valid_score = validation(model_acc, valid_loader)

print("Train:", train_score)
print("Valid:", valid_score)
print("Train:", train_score, flush=True, file=log_file)
print("Valid:", valid_score, flush=True, file=log_file)


Train: (0.96995, 0.08736814769208431)
Valid: (0.8735, 0.5040788244247436)


## Epoch average, alpha = 0.9

In [10]:
model = models.vgg16(pretrained=False)
model.classifier[6] = nn.Linear(4096, 10)
model.classifier.add_module("7", nn.LogSoftmax(-1))

model.to(device)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [11]:
validation(model, valid_loader)

(0.1036, 2.302524648284912)

In [12]:
log_file = open("vgg_log_augmentation_averaging_exp09.txt.no_resizing", "w")

In [13]:
optimizer = AcceleratedSGD(model.parameters(), 1e-2, k=10, momentum=0.9, weight_decay=1e-5, lambda_=1e-8, mode="epoch_avg", avg_alpha=0.9)
loss_fn = nn.NLLLoss()

In [14]:
epochs = 30

for epoch in range(epochs):
    print("Epoch", epoch+1)
    loss_log = []
    train_epoch(loss_log)
    print(f"Training loss: {np.mean(loss_log):.4f}")
    optimizer.finish_epoch()
    val_acc, val_loss = validation(model, valid_loader)
    print(f"Validation accuracy: {val_acc:.4f}, validation loss: {val_loss:.4f}")
    print("Epoch", epoch+1, 
          f"Training loss: {np.mean(loss_log):.4f}, validation accuracy: {val_acc:.4f}, validation loss: {val_loss:.4f}",
          file=log_file, flush=True
         )

Epoch 1



Training loss: 2.0756



Validation accuracy: 0.3218, validation loss: 1.7948
Epoch 2



Training loss: 1.7398



Validation accuracy: 0.4048, validation loss: 1.5608
Epoch 3


...


Validation accuracy: 0.8142, validation loss: 0.5843
Epoch 29



Training loss: 0.3332



Validation accuracy: 0.8373, validation loss: 0.5231
Epoch 30



Training loss: 0.3180



Validation accuracy: 0.8273, validation loss: 0.5681


In [15]:
train_score = validation(model, train_loader)
valid_score = validation(model, valid_loader)
print("Train:", train_score)
print("Valid:", valid_score)
print("Train:", train_score, flush=True, file=log_file)
print("Valid:", valid_score, flush=True, file=log_file)


Train: (0.892825, 0.3033597732543945)
Valid: (0.8273, 0.568075549030304)


In [16]:
model_acc = deepcopy(model)
optimizer.accelerate()
optimizer.store_parameters([model_acc.parameters()])

In [17]:
model_acc.to(device)
train_score = validation(model_acc, train_loader)
valid_score = validation(model_acc, valid_loader)

print("Train:", train_score)
print("Valid:", valid_score)
print("Train:", train_score, flush=True, file=log_file)
print("Valid:", valid_score, flush=True, file=log_file)


Train: (0.92575, 0.2214997649908066)
Valid: (0.8637, 0.4155295261383057)


In [18]:
optimizer.param_groups[0]["lr"] = 1e-3

In [19]:
epochs = 30

for epoch in range(epochs):
    print("Epoch", epoch+1)
    loss_log = []
    train_epoch(loss_log)
    print(f"Training loss: {np.mean(loss_log):.4f}")
    optimizer.finish_epoch()
    val_acc, val_loss = validation(model, valid_loader)
    print(f"Validation accuracy: {val_acc:.4f}, validation loss: {val_loss:.4f}")
    print("Epoch", epoch+1, 
          f"Training loss: {np.mean(loss_log):.4f}, validation accuracy: {val_acc:.4f}, validation loss: {val_loss:.4f}",
          file=log_file, flush=True
         )

Epoch 1



Training loss: 0.2171



Validation accuracy: 0.8632, validation loss: 0.4707
Epoch 2



Training loss: 0.1885



Validation accuracy: 0.8629, validation loss: 0.4697
Epoch 3


...


Validation accuracy: 0.8681, validation loss: 0.5475
Epoch 29



Training loss: 0.0904



Validation accuracy: 0.8703, validation loss: 0.5529
Epoch 30



Training loss: 0.0928



Validation accuracy: 0.8664, validation loss: 0.5629


In [20]:
train_score = validation(model, train_loader)
valid_score = validation(model, valid_loader)
print("Train:", train_score)
print("Valid:", valid_score)
print("Train:", train_score, flush=True, file=log_file)
print("Valid:", valid_score, flush=True, file=log_file)


Train: (0.9694, 0.08753704386949539)
Valid: (0.8664, 0.5629105701446533)


In [21]:
model_acc = deepcopy(model)
optimizer.accelerate()
optimizer.store_parameters([model_acc.parameters()])

In [22]:
model_acc.to(device)
train_score = validation(model_acc, train_loader)
valid_score = validation(model_acc, valid_loader)

print("Train:", train_score)
print("Valid:", valid_score)
print("Train:", train_score, flush=True, file=log_file)
print("Valid:", valid_score, flush=True, file=log_file)


Train: (0.9708, 0.08238556798696518)
Valid: (0.87, 0.5281442687988281)


In [ ]:
exit